In [ ]:
using Plots
using Interact
using Combinatorics
using StatsBase
using LinearAlgebra

In [ ]:
function possible_squares(G)
    M, N = size(G)
    squares = Tuple{Int,Int}[]  
    
    for ii = 1:M
        for jj = 1:N
            if G[ii, jj] == 0
                if  (ii == 1 && jj > 1 && G[ii, jj-1] == 1) ||
                    (jj == 1 && ii > 1 && G[ii-1, jj] == 1) ||
                    (ii > 1 && jj > 1 && G[ii-1, jj] == 1 && G[ii, jj-1] == 1)
                    push!(squares, (ii, jj))
                end
            end
        end
    end
    
    return squares
end

In [ ]:
const possible = 0.4
const successful = 0.6

function calculate_random_growth(M, N, q, T=5000)

    G = zeros(M, N)

    all_heatmaps = typeof(G)[]

    # initial condition: bottom left square on
    G[1, 1] = true  
    push!(all_heatmaps, copy(G))

    for t = 1:T
        squares = possible_squares(G)
		dist = Array{Float64}(undef,length(squares))
		index = reverse(1:length(squares))
		i = 1
        for square in squares
			dist[i] = square[1]+square[2]
			i += 1
		end	
		sorted =  sort(index,by=x->dist[x])
        # highlight the possible squares:
        for square in squares
            G[square[1], square[2]] = possible
        end

        push!(all_heatmaps, copy(G))

		i = 1
        # choose squares to actually grow and highlight them:
        	for square in squares
            	if rand() > q   # grows with probability p = 1 - q
                	G[square[1], square[2]] = successful
           	   end
        	end

		#choose a single square
			#square = sample(squares,Weights)
			#G[square[1],square[2]] = successful

        push!(all_heatmaps, copy(G))
        
        # finish growth:
        for i in 1:M
            for j in 1:N
                if G[i, j] == successful
                    G[i, j] = 1    # complete growth
                    
                elseif G[i, j] == possible
                    G[i, j] = 0    # square dies if growth doesn't happen
                end
            end
        end
        
        push!(all_heatmaps, copy(G))

        # vectorized version:
#         G[G .== 0.5] .= 1
#         G[G .== 0.25] .= 0

    end

    return all_heatmaps
end

In [ ]:
function visualize_random_growth(M, N, q, T=100)
    
    all_heatmaps = calculate_random_growth(M, N, q, T)

	i = length(all_heatmaps)
        
        t = (i - 1) ÷ 3 
        
        # 3 slider movements for each time step
        
        p = heatmap(all_heatmaps[i], aspect_ratio=1, colorbar=false, clims=(0, 1), 
                    xlims = (0, N), ylims=(0, M))
        
        # p = contour(all_heatmaps[i], levels=[1])
        
        plot!(title="t = $t")
        
        range = t .* (0:0.001:(1-q))        
        
        p

	
    
end

In [ ]:
visualize_random_growth(500, 500, 0.5, 1000)
visualize_random_growth(500, 500, 0.01, 1000)

In [12]:
#Now, let us make the structure that of a DPP

In [31]:
function dpp_sample(n,L)
	prob = Array{Float64}(undef,length([i for i in combinations(1:n)]))
	subset = Array{Vector{Int64}}(undef,length([i for i in combinations(1:n)]))
	for j in  1:length([i for i in combinations(1:n)])
		subset[j] = [i for i in combinations(1:n)][j]
		prob[j] = det(L[[i for i in combinations(1:n)][j],[i for i in combinations(1:n)][j]])/det(I+L)
	end
	subset = append!(subset,Float64[])
	prob = append!(prob,1/det(I+L))
	return sample(subset,Weights(prob))
end	


dpp_sample (generic function with 1 method)

In [32]:
function calculate_random_growth_dpp(M, N, T=5000)

	# input L matrix as needed
	U = randn(max(M,N),max(M,N))
	L = U*U'
    G = zeros(M, N)

    all_heatmaps = typeof(G)[]

    # initial condition: bottom left square on
    G[1, 1] = true  
    push!(all_heatmaps, copy(G))

    for t = 1:T
        squares = possible_squares(G)
        
        # highlight the possible squares:
        for square in squares
            G[square[1], square[2]] = possible
        end

        push!(all_heatmaps, copy(G))


        # choose squares to actually grow and highlight them:
        n = length(squares)
		for square in squares[dpp_sample(n,L)]
			G[square[1], square[2]] = successful
		end

        push!(all_heatmaps, copy(G))
        
        # finish growth:
        for i in 1:M
            for j in 1:N
                if G[i, j] == successful
                    G[i, j] = 1    # complete growth
                    
                elseif G[i, j] == possible
                    G[i, j] = 0    # square dies if growth doesn't happen
                end
            end
        end
        
        push!(all_heatmaps, copy(G))

        # vectorized version:
#         G[G .== 0.5] .= 1
#         G[G .== 0.25] .= 0

    end

    return all_heatmaps
end

calculate_random_growth_dpp (generic function with 2 methods)

In [33]:
function visualize_random_growth_dpp(M, N, T=100)

    all_heatmaps = calculate_random_growth_dpp(M, N, T)

	i = length(all_heatmaps)
        
        t = (i - 1) ÷ 3 
        
        # 3 slider movements for each time step
        
        p = heatmap(all_heatmaps[i], aspect_ratio=1, colorbar=false, clims=(0, 1), 
                    xlims = (0, N), ylims=(0, M))
        
        # p = contour(all_heatmaps[i], levels=[1])
        
        plot!(title="t = $t")
        
        range = t .* (0:0.001:1)        
        
        p

	
    
end


visualize_random_growth_dpp (generic function with 2 methods)